In [1]:
import pandas as pd
import numpy as np

In [3]:
lab_sofa=pd.read_csv("Labs_withSOFA.csv")
vent_sofa=pd.read_csv("drugate_norm_updated.csv",usecols=['patientunitstayid','offset','SOFA_cardio'])
gcs_sofa=pd.read_csv("gcs_withSOFA.csv",usecols=['patientunitstayid','offset','SOFA_GCS'])
t_sus=pd.read_csv("ABX_BC_pid_offset_Clean_TsuspicionMax.csv")

In [4]:
lab_sofa.head()

,patientunitstayid,offset,SOFA_Coagulation,SOFA_Liver,SOFA_Respiration,SOFA_Renal
0,141168,516.0,0.0,2.0,0.0,0.0
1,141168,1133.0,0.0,2.0,0.0,2.0
2,141168,1701.0,0.0,0.0,0.0,4.0
3,141168,1805.0,0.0,0.0,2.0,0.0
4,141168,2010.0,0.0,0.0,2.0,0.0


In [5]:
vent_sofa.head()

,patientunitstayid,SOFA_cardio,offset
0,242082,4.0,425
1,242082,3.0,125
2,242082,3.0,55
3,242082,3.0,665
4,242082,3.0,230


In [6]:
gcs_sofa.head()

,patientunitstayid,offset,SOFA_GCS
0,141924,2109,0.0
1,141924,1894,0.0
2,141924,424,0.0
3,141944,484,0.0
4,141944,2389,1.0


In [7]:
t_sus.head()

,patientunitstayid,max
0,242040,188.0
1,242154,8589.0
2,242290,165.0
3,242319,1221.0
4,242434,1.0


In [8]:
patientIDs_IV = t_sus.patientunitstayid.unique().tolist()
patientIDs_IV_sub = pd.DataFrame(patientIDs_IV)
patientIDs_IV_sub.columns=['patientunitstayid']

Let's start the merging

In [23]:
labs_sus=pd.merge(lab_sofa,patientIDs_IV_sub,how="inner",on='patientunitstayid').drop_duplicates()
vent_sus=pd.merge(vent_sofa,patientIDs_IV_sub,how="inner",on='patientunitstayid').drop_duplicates()
gcs_sofa_sus=pd.merge(gcs_sofa,patientIDs_IV_sub,how="inner",on='patientunitstayid').drop_duplicates()
labs_vent=pd.merge(labs_sus,vent_sus,on=['patientunitstayid','offset'],how="outer").drop_duplicates()
final_sofa=pd.merge(labs_vent,gcs_sofa_sus,on=['patientunitstayid','offset'],how="outer").drop_duplicates()
final_sofa=final_sofa.fillna(0)
final_sofa['Total_SOFA']=final_sofa['SOFA_Coagulation']+final_sofa['SOFA_Liver']+final_sofa['SOFA_Respiration']+final_sofa['SOFA_Renal']+final_sofa['SOFA_cardio']+final_sofa['SOFA_GCS']
del lab_sofa
del vent_sofa
del gcs_sofa
del t_sus

In [28]:
final_sofa.groupby(['patientunitstayid']).nunique()

,patientunitstayid,offset,SOFA_Coagulation,SOFA_Liver,SOFA_Respiration,SOFA_Renal,SOFA_cardio,SOFA_GCS,Total_SOFA
patientunitstayid,,,,,,,,,
242040,1,8,1,1,1,2,1,3,4
242154,1,81,1,1,2,1,1,5,5
242290,1,20,2,1,1,2,1,4,5
242319,1,11,3,1,1,3,2,3,5
242434,1,9,1,1,1,1,1,4,4
242544,1,23,2,1,2,1,1,3,5
242851,1,14,1,1,1,2,1,3,4
243208,1,8,1,1,1,2,1,3,5
243334,1,36,1,1,1,2,2,3,3


In [25]:
final_sofa.head()

,patientunitstayid,offset,SOFA_Coagulation,SOFA_Liver,SOFA_Respiration,SOFA_Renal,SOFA_cardio,SOFA_GCS,Total_SOFA
0,242040,27.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,242040,1061.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,242040,2567.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,242040,3952.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
4,242040,4757.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
